In [ ]:
import osmnx as ox
import networkx as nx
import folium
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

# Load the hotspots dataset
hotspots_df = pd.read_csv('hotspots.csv')

# Ask for user input for source and destination
source_location = input("Enter the source location: ")  # e.g., "Telegraph Hill"
destination_location = input("Enter the destination location: ")  # e.g., "Mount Davidson"

In [ ]:
# Geocode the input locations to get their coordinates
source_coords = ox.geocode(source_location + ', San Francisco, California, USA')
destination_coords = ox.geocode(destination_location + ', San Francisco, California, USA')

# Create a street network graph for San Francisco
G = ox.graph_from_place('San Francisco, California, USA', network_type='drive')

# Find the nearest nodes for the source and destination
source_node = ox.distance.nearest_nodes(G, source_coords[1], source_coords[0])
destination_node = ox.distance.nearest_nodes(G, destination_coords[1], destination_coords[0])

# Compute the shortest path before removing hotspots
shortest_path_before = nx.shortest_path(G, source_node, destination_node, weight='length')

# Find the nearest nodes in the street network for the hotspots
hotspot_nodes = [ox.distance.nearest_nodes(G, lon, lat) for lat, lon in zip(hotspots_df['Latitude'], hotspots_df['Longitude'])]

# Remove the hotspot nodes from the graph
G_modified = G.copy()
for node in hotspot_nodes:
    if node in G_modified:
        G_modified.remove_node(node)

# Compute the shortest path after removing hotspots
shortest_path_after = nx.shortest_path(G_modified, source_node, destination_node, weight='length')


In [ ]:
# Create map visualization using Folium
m = folium.Map(location=[(source_coords[0] + destination_coords[0]) / 2, 
                        (source_coords[1] + destination_coords[1]) / 2], 
                zoom_start=12)

# Add the source, destination, and paths to the map
folium.Marker(source_coords, popup=source_location, icon=folium.Icon(color='green')).add_to(m)
folium.Marker(destination_coords, popup=destination_location, icon=folium.Icon(color='red')).add_to(m)
ox.plot_route_folium(G, shortest_path_before, route_map=m, color='blue')
ox.plot_route_folium(G_modified, shortest_path_after, route_map=m, color='red')

# Add hotspots as markers to the map
for lat, lon in zip(hotspots_df['Latitude'], hotspots_df['Longitude']):
    folium.Marker([lat, lon], popup='Hotspot', icon=folium.Icon(color='lightgray')).add_to(m)
    
# Save the map to an HTML file
# m.save("sf_shortest_path_comparison.html")

In [ ]:
m